![image](competi.png)

#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

In [69]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor


In [70]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("diamonds_train.csv", index_col=0)

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40345 entries, 0 to 40344
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40345 non-null  float64
 1   cut      40345 non-null  object 
 2   color    40345 non-null  object 
 3   clarity  40345 non-null  object 
 4   depth    40345 non-null  float64
 5   table    40345 non-null  float64
 6   x        40345 non-null  float64
 7   y        40345 non-null  float64
 8   z        40345 non-null  float64
 9   price    40345 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 3.4+ MB


In [72]:
df.color.value_counts()

G    8499
E    7351
F    7130
H    6234
D    5074
I    4023
J    2034
Name: color, dtype: int64

In [73]:
encoder = OrdinalEncoder(categories=[["J", "I", "H", "G", "F","E", "D",]])
encoder.fit(df[["color"]])
df["color"] = encoder.transform(df[["color"]])
df.clarity.value_counts()
encoder = OrdinalEncoder(categories=[["I1", "SI2", "SI1", "VS1", "VS2","VVS2", "VVS1","IF"]])
encoder.fit(df[["clarity"]])
df["clarity"] = encoder.transform(df[["clarity"]])
df.cut.value_counts()
encoder = OrdinalEncoder(categories=[["Fair", "Good", "Very Good", "Premium", "Ideal"]])
encoder.fit(df[["cut"]])
df["cut"] = encoder.transform(df[["cut"]])

In [74]:
df["cut"]

0        4.0
1        2.0
2        3.0
3        4.0
4        1.0
        ... 
40340    3.0
40341    4.0
40342    2.0
40343    2.0
40344    4.0
Name: cut, Length: 40345, dtype: float64

In [75]:
df.corr()

,carat,cut,color,clarity,depth,table,x,y,z,price
carat,1.000000,-0.133226,-0.285737,-0.346642,0.029414,0.181273,0.975077,0.944445,0.967289,0.921887
cut,-0.133226,1.000000,0.020164,0.188360,-0.216858,-0.435731,-0.124068,-0.118789,-0.149181,-0.052736
color,-0.285737,0.020164,1.000000,-0.003954,-0.048637,-0.022557,-0.263915,-0.255262,-0.266589,-0.166340
clarity,-0.346642,0.188360,-0.003954,1.000000,-0.067222,-0.154561,-0.365112,-0.350003,-0.365825,-0.143240
depth,0.029414,-0.216858,-0.048637,-0.067222,1.000000,-0.288987,-0.023389,-0.027880,0.097399,-0.009228
table,0.181273,-0.435731,-0.022557,-0.154561,-0.288987,1.000000,0.193986,0.180449,0.153175,0.127338
x,0.975077,-0.124068,-0.263915,-0.365112,-0.023389,0.193986,1.000000,0.966827,0.985080,0.883163
y,0.944445,-0.118789,-0.255262,-0.350003,-0.027880,0.180449,0.966827,1.000000,0.959957,0.857564
z,0.967289,-0.149181,-0.266589,-0.365825,0.097399,0.153175,0.985080,0.959957,1.000000,0.872855
price,0.921887,-0.052736,-0.166340,-0.143240,-0.009228,0.127338,0.883163,0.857564,0.872855,1.000000


In [76]:
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z',
       'price'],
      dtype='object')

In [77]:

X = np.array(df[["carat","cut","clarity","table","color","depth","x","y","z"]])
y = np.array(df["price"])
X.shape


(40345, 9)

In [78]:
y.shape

(40345,)

### 2. Dividir X_train, X_test, y_train, y_test

In [79]:
# el test_size 0.20 y el random_state puede cambiar
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=7)

In [80]:
train_X

array([[1.09, 3.  , 1.  , ..., 6.75, 6.61, 4.07],
       [1.5 , 3.  , 2.  , ..., 7.27, 7.34, 4.58],
       [0.41, 4.  , 3.  , ..., 4.77, 4.8 , 2.97],
       ...,
       [0.34, 3.  , 5.  , ..., 4.59, 4.54, 2.72],
       [0.5 , 0.  , 5.  , ..., 4.95, 4.84, 3.31],
       [0.91, 1.  , 5.  , ..., 6.06, 6.1 , 3.9 ]])

### 3. Asignar el modelo (vacío) a una variable
Aquí meteríais los parámetros. 

**Consejo**: Usa GridSearch y vuélvete loca o loco probando modelos e hiperparámetros.

In [142]:
rf = RandomForestRegressor(n_estimators=100, random_state=7,min_impurity_decrease=10,min_weight_fraction_leaf=0.1)
rf.fit(train_X, train_y)


RandomForestRegressor(min_impurity_decrease=10, min_weight_fraction_leaf=0.1,
                      random_state=7)

In [143]:
predictions = rf.predict(test_X)

In [144]:
errors = abs(predictions - test_y)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 830.38 degrees.


In [145]:
np.sqrt(mean_squared_error(test_y, predictions))

1498.4041601404183

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

_Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `diamonds_test.csv` nos arriesgamos a que digamos, no salga lo esperado._

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `diamonds_train.csv` al completo.

**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**

Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `diamonds_test.csv` para predecir.

**¿De dónde saco `diamonds_test.csv`?**

In [62]:
X_pred = pd.read_csv("diamonds_test.csv", index_col = 0)
X_pred.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.30,Ideal,H,SI2,60.0,56.0,4.41,4.43,2.65
1,0.34,Ideal,D,IF,62.1,57.0,4.52,4.46,2.79
2,1.57,Very Good,I,VS2,60.3,58.0,7.58,7.55,4.56
3,0.31,Ideal,H,VS2,61.8,57.0,4.32,4.36,2.68
4,1.51,Good,I,VVS1,64.0,60.0,7.26,7.21,4.63


In [63]:
encoder = OrdinalEncoder(categories=[["J", "I", "H", "G", "F","E", "D",]])
encoder.fit(X_pred[["color"]])
X_pred["color"] = encoder.transform(X_pred[["color"]])
X_pred.clarity.value_counts()
encoder = OrdinalEncoder(categories=[["I1", "SI2", "SI1", "VS1", "VS2","VVS2", "VVS1","IF"]])
encoder.fit(X_pred[["clarity"]])
X_pred["clarity"] = encoder.transform(X_pred[["clarity"]])
df.cut.value_counts()
encoder = OrdinalEncoder(categories=[["Fair", "Good", "Very Good", "Premium", "Ideal"]])
encoder.fit(X_pred[["cut"]])
X_pred["cut"] = encoder.transform(X_pred[["cut"]])

In [64]:
X_pred = np.array(X_pred[["carat","cut","clarity","color","table","depth",'x', 'y', 'z']])

In [65]:
X.shape

(40345, 9)

In [66]:
X_pred.shape

(13449, 9)

**¿Por qué da error?** 

In [67]:
predictions_submit = rf.predict(X_pred)
predictions_submit

array([  428.676,  1729.114,  9289.742, ..., 15110.374, 10538.88 ,
         731.112])

In [68]:
X_pred

array([[0.3 , 4.  , 1.  , ..., 4.41, 4.43, 2.65],
       [0.34, 4.  , 7.  , ..., 4.52, 4.46, 2.79],
       [1.57, 2.  , 4.  , ..., 7.58, 7.55, 4.56],
       ...,
       [1.5 , 3.  , 4.  , ..., 7.3 , 7.37, 4.53],
       [1.7 , 3.  , 1.  , ..., 7.79, 7.68, 4.66],
       [0.3 , 4.  , 4.  , ..., 4.29, 4.34, 2.64]])

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [69]:
sample = pd.read_csv("sample_submission.csv")

In [70]:
sample.head()

,id,price
0,0,12132
1,1,11786
2,2,14684
3,3,15425
4,4,6724


In [71]:
sample.shape

(13449, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [72]:
submission = pd.DataFrame({"id": range(len(predictions_submit)), "price": predictions_submit})

In [73]:
submission.head()

,id,price
0,0,428.676
1,1,1729.114
2,2,9289.742
3,3,489.822
4,4,9296.564


In [74]:
submission.shape

(13449, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [75]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [76]:
help(chequeator)

Help on function chequeator in module __main__:

chequeator(df_to_submit)
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.



In [103]:
chequeator(submission)

You're ready to submit!
